MIDA Gondara and Wang(2018) in Python (using PyTorch)
https://arxiv.org/abs/1705.02737
https://gist.github.com/lgondara/18387c5f4d745673e9ca8e23f3d7ebd3 

### Note: Section 1 has been tested, moved to utils.py

# 1. Loading Dataset

## 1.1. Load a dataset and introduce missingness

Dataset used: Shuttle Dataset (https://archive.ics.uci.edu/ml/datasets/Statlog+(Shuttle)

### 1.1.1. Load the dataset and store it as dataframe(numeric)

In [1]:
import pandas as pd
import utils

In [2]:
#Test
filename = "data/shuttle/shuttle_trn"
train_df = utils.get_dataframe_from_csv(filename).iloc[:,:-1]  #remove label

INFO:root:Input filename has to be space separated data


In [3]:
#Test
filename = "data/shuttle/shuttle_trn_debug"
train_df = utils.get_dataframe_from_csv(filename).iloc[:,:-1]  #remove label

INFO:root:Input filename has to be space separated data


In [4]:
train_df.head()

,0,1,2,3,4,5,6,7,8
0,50,21,77,0,28,0,27,48,22
1,55,0,92,0,0,26,36,92,56
2,53,0,82,0,52,-5,29,30,2
3,37,0,76,0,28,18,40,48,8
4,37,0,79,0,34,-26,43,46,2


### 1.1.2. Inducing missingness

After dataset loading, start with inducing missingness. 

To start off, introduce simple random missing patterns (Missing Completely At Random), i.e. sample half of the variables and set observations in those variables to missing if an appended random uniform vector has value less than a certain threshhold. WIth threshold of 0.2, the procedure should introduce about 20% missingness.

In [5]:
#test
df1 = train_df[:]
df2 = utils.induce_missingness(df1,logger_level=20)

INFO:root: Returning new dataframe with missingness(MCAR) induced
INFO:root: Percentage of NaNs in returned dataframe : 2.22


In [6]:
df1.head()

,0,1,2,3,4,5,6,7,8
0,50,21,77,0,28,0,27,48,22
1,55,0,92,0,0,26,36,92,56
2,53,0,82,0,52,-5,29,30,2
3,37,0,76,0,28,18,40,48,8
4,37,0,79,0,34,-26,43,46,2


In [7]:
df2.head()

,0,1,2,3,4,5,6,7,8
0,50,21,77,0,NaN,0,NaN,NaN,NaN
1,55,0,92,0,0.0,26,36.0,92.0,56.0
2,53,0,82,0,52.0,-5,29.0,30.0,2.0
3,37,0,76,0,28.0,18,40.0,48.0,8.0
4,37,0,79,0,34.0,-26,43.0,46.0,2.0


### 1.1.3. Create Train-Test split

Create 70% training data and 30%  test data which includes missingness and a test data without missingness so we can calculate performance. 

In [8]:
#Test
a,b,c = utils.create_train_test_split(df1)
print(a.head())
print(b.head())
print(c.head())

INFO:root: Returning new dataframe with missingness(MCAR) induced
INFO:root: Percentage of NaNs in returned dataframe : 2.22
INFO:root: Returning train_df, test_df, full_test_df after splitting dataframe in 0.7/0.3 split 
INFO:root: Note: full_test_df is the same as test_df but without NaNs


     0  1    2  3     4   5     6     7     8
6   56  0   81  0  -4.0  11  25.0  86.0  62.0
7   55 -1   95 -3  54.0  -4  40.0  41.0   2.0
9   37  0  101 -7  28.0   0  64.0  73.0   8.0
18  49  3   82  0  50.0   4  33.0  33.0   0.0
1   55  0   92  0   0.0  26  36.0  92.0  56.0
     0    1    2  3     4   5     6     7    8
3   37    0   76  0  28.0  18  40.0  48.0  8.0
4   37    0   79  0  34.0 -26  43.0  46.0  2.0
19  37 -318  105 -2  36.0  -4  68.0  69.0  0.0
0   50   21   77  0   NaN   0   NaN   NaN  NaN
12  38    2   77  0  38.0   7  39.0  38.0  0.0
     0    1    2  3   4   5   6   7   8
3   37    0   76  0  28  18  40  48   8
4   37    0   79  0  34 -26  43  46   2
19  37 -318  105 -2  36  -4  68  69   0
0   50   21   77  0  28   0  27  48  22
12  38    2   77  0  38   7  39  38   0


# 2. Modelling

Proceed to modelling.

In R:
Start with initializing 'h2o' package and then reading the training and test datasets as the 'h2o's supported format.
Then run imputation model multiple times as each new start would initialize the weights with different values.<br>
Info at: <br>
[h2o](https://cran.r-project.org/web/packages/h2o/h2o.pdf) package offers an easy to use function for implementing autoencoders. 
More information is available at this [link](http://docs.h2o.ai/h2o/latest-stable/h2o-docs/booklets/DeepLearningBooklet.pdf).

In Python:

In [9]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.functional as F

In [10]:
#Settings for device, randomization seed, default tensor type, kwargs for memory #DevSeedTensKwargs
RANDOM_SEED = 18
np.random.seed(RANDOM_SEED)

if torch.cuda.is_available():
    device = 'cuda'
    torch.cuda.manual_seed(RANDOM_SEED)
    torch.set_default_tensor_type(torch.cuda.FloatTensor)
    kwargs = {'num_workers':4, 'pin_memory' :True}
else:
    device = 'cpu'
    torch.manual_seed(RANDOM_SEED)
    torch.set_default_tensor_type(torch.FloatTensor)
    kwards = {}

In [11]:
import dataset_module

In [ ]:
# from importlib import reload
# reload(dataset_module)

In [12]:
df2.head()

,0,1,2,3,4,5,6,7,8
0,50,21,77,0,NaN,0,NaN,NaN,NaN
1,55,0,92,0,0.0,26,36.0,92.0,56.0
2,53,0,82,0,52.0,-5,29.0,30.0,2.0
3,37,0,76,0,28.0,18,40.0,48.0,8.0
4,37,0,79,0,34.0,-26,43.0,46.0,2.0


In [13]:
trainset = dataset_module.DataSetForImputation(df2, normalize=False)  #normalize True for [0,1] normalization for dataframe

In [14]:
trainset

Dataframe Size:20, Perc of NaNs: 2.22

In [15]:
trainset[0]

(tensor([50.0000, 21.0000, 77.0000,  0.0000, 30.2105,  0.0000, 38.8947, 55.3684,
         16.5263]),
 tensor([50.0000, 21.0000, 77.0000,  0.0000, 30.2105,  0.0000, 38.8947, 55.3684,
         16.5263]))

In [16]:
trainset[1]

(tensor([55.,  0., 92.,  0.,  0., 26., 36., 92., 56.]),
 tensor([55.,  0., 92.,  0.,  0., 26., 36., 92., 56.]))

In [17]:
trainset[2]

(tensor([53.,  0., 82.,  0., 52., -5., 29., 30.,  2.]),
 tensor([53.,  0., 82.,  0., 52., -5., 29., 30.,  2.]))

In [18]:
import Modelling
net = Modelling.DenoisingAutoEncoder(len(trainset.variables()))

In [19]:
net

DenoisingAutoEncoder(
  (drop_layer): Dropout(p=0.5)
  (linear_layer_list): ModuleList(
    (0): Linear(in_features=9, out_features=16, bias=True)
    (1): Linear(in_features=16, out_features=23, bias=True)
    (2): Linear(in_features=23, out_features=30, bias=True)
    (3): Linear(in_features=30, out_features=23, bias=True)
    (4): Linear(in_features=23, out_features=16, bias=True)
    (5): Linear(in_features=16, out_features=9, bias=True)
  )
)

# 3. Training

1. Increased batch size from 16 to 256
2. No normalization
3. Removed /VARIABLES in RMSE

In [20]:
import torch.utils.data as td
from torch.optim import Adam

LR = 1e-3
BATCH_SIZE = 256  #16 wasn't working too well even after 600 iterations
VARIABLES  = len(trainset.variables()) #9
DATAPOINTS = len(trainset)  #45600

import Modelling
net = Modelling.DenoisingAutoEncoder(len(trainset.variables()))

criterion = nn.MSELoss()
net = net.to(device) 

train_loader = td.DataLoader(trainset, batch_size= BATCH_SIZE, shuffle= True, **kwargs) 
optimizer = Adam(net.parameters(), lr = LR)

LOG_INTERVAL = 2
SAVE_INTERVAL = 50

In [22]:
'''
TO DO:
:- Normalization between 0 and 1 - Error blowing up for some reason
:- (0,1) Normalization for better convergence - how to handle this elegantly while testing, because this has been trained for something between 0 and 1
:- Nesterov Momentum + Adam- Pytorch? Decay factor?
'''

from tqdm import tqdm_notebook as tqdm
def train(start_steps = 0, end_steps = 5, net=None, logger_level = 10):
    import logging
    logger = logging.getLogger()
    logger.setLevel(logger_level)
    
    agg_loss = 0.0
    for epoch in tqdm(range(start_steps, end_steps)):
        count = epoch-start_steps+1
        net.train()
        #Epoch begins
        for x, d in tqdm(train_loader):
#             x = x/x.sum(0).expand_as(x)  
#             d = d/d.sum(0).expand_as(d)  # Normalize between [0,1] for better convergence 
            optimizer.zero_grad()
            x = x.to(device)
            with torch.no_grad():
                d = d.to(device)
            y = net(x)
            loss = torch.sqrt(criterion(y, d))   #RMSE Loss   
            loss.backward()
            optimizer.step()
            agg_loss += loss.item()
            logging.debug(loss)
            
        if count%LOG_INTERVAL == 0:
            print(f"Epoch number:{epoch}  Aggregate loss: {agg_loss/(LOG_INTERVAL):.2f}")  #Determine the correct interval and value for loss
            agg_loss= 0.0
            
        if epoch%SAVE_INTERVAL== 0:
            torch.save(net.state_dict(),f"./artifacts/DAE_saved_model_epoch{epoch}")
        #Epoch Ends

In [27]:
train(10,1000, net, logger_level=20)

Epoch number:11  Aggregate loss: 49.43


Epoch number:13  Aggregate loss: 49.41


Epoch number:15  Aggregate loss: 49.39


Epoch number:17  Aggregate loss: 49.36


Epoch number:19  Aggregate loss: 49.33


...

Epoch number:991  Aggregate loss: 39.39


Epoch number:993  Aggregate loss: 39.38


Epoch number:995  Aggregate loss: 39.36


Epoch number:997  Aggregate loss: 39.35


Epoch number:999  Aggregate loss: 39.33



Ran for 500 iterations <br>
Observation - Bad performance : Maybe quickly getting stuck at plateaus of local minima? <br>
              Soln: Maybe need a smaller learning rate after a while? DONE <br>
              Soln: Check paper to see if all the features have been implemented exactly (like Nesterov?)
              
Problems/Issues :


Next steps:
Read the paper again, check the exact implementation of each thing - make a list of things that you don't exactly understand - Nesterov Momentum,

1. WHY IS THE MODEL PREDICTING SAME OUTPUT FOR EVERY INPUT?

Tried for a smaller dataset, Values are again identically predicted for all input ? Why is it so?

Next : How to debug a neural net, how to 
2. Why is my loss value so large? Shouldn't it get normalized when I am dividing by LOG_INTERVAL?
2. How do I normalize between 0 and 1?
3. What to print exactly and where?
3. Possibly save optimizer parameters? 

#torch.save(optimizer.state_dict(), filename)
#optimizer.load_state_dict(torch.load(filename))

In [ ]:
#Tweaking the learning rate to improve convergence speed
optimizer = Adam(net.parameters(), LR)

In [ ]:
# Modelling Loading from saved point

# model =  Modelling.DenoisingAutoEncoder(len(trainset.variables()))
# model.load_state_dict(torch.load("./artifacts/saved_model_epoch50"))
# model.eval()

In [ ]:
torch.set_printoptions(sci_mode=False)

In [24]:
net = net.eval()

print((net(trainset[0][0]).detach()))
print(trainset[0][0])

tensor([-0.1397,  0.3450,  0.4168, -0.2188,  0.2542, -0.5428,  0.2025,  0.3588,
         0.3152])
tensor([50.0000, 21.0000, 77.0000,  0.0000, 30.2105,  0.0000, 38.8947, 55.3684,
        16.5263])


In [28]:
net = net.eval()

print((net(trainset[0][0]).detach()))
print(trainset[0][0])

tensor([ 17.5230, -12.0658,  18.8017,  -0.9500,  17.1670,   1.0000,  17.4621,
         18.8654,  14.4723])
tensor([50.0000, 21.0000, 77.0000,  0.0000, 30.2105,  0.0000, 38.8947, 55.3684,
        16.5263])


In [ ]:
net = net.eval()

print((net(trainset[0][0]).detach()))
print(trainset[0][0])

In [25]:
print(net(trainset[5][0]).detach())
print(trainset[5][0])

tensor([-0.1742,  0.3580,  0.4133, -0.1918,  0.1839, -0.5554,  0.1205,  0.3028,
         0.3152])
tensor([85.,  0., 88., -4.,  6.,  1.,  3., 83., 80.])


In [29]:
print(net(trainset[5][0]).detach())
print(trainset[5][0])

tensor([ 17.5230, -12.0658,  18.8017,  -0.9500,  17.1670,   1.0000,  17.4621,
         18.8654,  14.4723])
tensor([85.,  0., 88., -4.,  6.,  1.,  3., 83., 80.])


In [26]:
print(net(trainset[15][0]).detach())
print(trainset[15][0])

tensor([-0.1394,  0.3406,  0.4029, -0.2072,  0.2592, -0.5284,  0.2117,  0.3608,
         0.3173])
tensor([ 41.,   0.,  89.,   1.,  38., -16.,  48.,  50.,   2.])


In [ ]:
# torch.set_printoptions(sci_mode=True)

In [ ]:
train(500,510, net,  logger_level=20)